# Imports and setup

In [14]:
import dlib
import cv2
import os
import numpy as np
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
from PIL import Image
import io
import html
import time
import torchvision.datasets as dset
from torchvision import transforms
import torchvision.transforms as transforms
from torchvision.transforms.functional  import InterpolationMode
import torchvision
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import statistics
from statistics import mean
import matplotlib.pyplot as plt
import numpy as np

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
# Define paths

data_dir = '/content/drive/MyDrive/GAN_Project/To_Sumbit/dataset_class/'  # Data directory
checkpoint_dir = '/content/drive/MyDrive/GAN_project/checkpoints/class_model/MyDataset_cyclegan'  # Checkpoints directory

In [17]:
# Clone the repository

!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix
%cd pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2516 (delta 0), reused 1 (delta 0), pack-reused 2513
Receiving objects: 100% (2516/2516), 8.20 MiB | 27.63 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.
/content/pytorch-CycleGAN-and-pix2pix/pytorch-CycleGAN-and-pix2pix


In [18]:
# Install dependencies
!pip install -r requirements.txt

In [19]:
# Create symbolic links to your dataset
!mkdir -p datasets/MyDataset
!ln -s /content/drive/MyDrive/GAN_Project/To_Sumbit/dataset_class/without_hat_class datasets/MyDataset/trainA
!ln -s /content/drive/MyDrive/GAN_Project/To_Sumbit/dataset_class/with_hat_class datasets/MyDataset/trainB

!ln -s /content/drive/MyDrive/GAN_Project/To_Sumbit/dataset_class/test_without_hat datasets/MyDataset/testA
!ln -s /content/drive/MyDrive/GAN_Project/To_Sumbit/dataset_class/test_with_hat datasets/MyDataset/testB




In [20]:

# Verify the directories
print("trainA directory contents:")
!ls -al ./datasets/MyDataset/trainA

print("trainB directory contents:")
!ls -al ./datasets/MyDataset/trainB

# Check if symbolic links are created correctly
print("\nSymbolic link for trainA:")
!readlink ./datasets/MyDataset/trainA

print("\nSymbolic link for trainB:")
!readlink ./datasets/MyDataset/trainB

trainA directory contents:
lrwxrwxrwx 1 root root 76 Jun 27 11:19 ./datasets/MyDataset/trainA -> /content/drive/MyDrive/GAN_Project/To_Sumbit/dataset_class/without_hat_class
trainB directory contents:
lrwxrwxrwx 1 root root 73 Jun 27 11:19 ./datasets/MyDataset/trainB -> /content/drive/MyDrive/GAN_Project/To_Sumbit/dataset_class/with_hat_class

Symbolic link for trainA:
/content/drive/MyDrive/GAN_Project/To_Sumbit/dataset_class/without_hat_class

Symbolic link for trainB:
/content/drive/MyDrive/GAN_Project/To_Sumbit/dataset_class/with_hat_class


In [21]:
import shutil

In [22]:
!pip install dominate

In [23]:

import dominate

# Train

In [24]:
!python train.py --dataroot ./datasets/MyDataset --name MyDataset_cyclegan --model cycle_gan --n_epochs 10 --n_epochs_decay 190 --save_epoch_freq 5 --continue_train --checkpoints_dir /content/drive/MyDrive/GAN_project/checkpoints/class_model --batch_size 16 --epoch latest --lr 0.0002


----------------- Options ---------------
               batch_size: 16                            	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: /content/drive/MyDrive/GAN_project/checkpoints/class_model	[default: ./checkpoints]
           continue_train: True                          	[default: False]
                crop_size: 256                           
                 dataroot: ./datasets/MyDataset          	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 256                 

# Display imgs per epochs

In [ ]:
from IPython.display import display, HTML, Image
import glob
import os
import base64

# Path to the directory containing generated images
result_dir = '/content/drive/MyDrive/GAN_project/checkpoints/class_model/MyDataset_cyclegan/web/images'

# Group images by epoch
images_by_epoch = {}
for img_path in glob.glob(f"{result_dir}/*.png"):
    image_name = os.path.basename(img_path)
    epoch = image_name.split('_')[0]
    if epoch not in images_by_epoch:
        images_by_epoch[epoch] = []
    if 'fake' in image_name or 'real' in image_name:
        images_by_epoch[epoch].append(img_path)

# Display images grouped by epoch
for epoch, images in sorted(images_by_epoch.items()):
    display(HTML(f'<h3>Epoch {epoch}</h3>'))
    html = '<div style="display: flex; flex-wrap: wrap;">'
    for img_path in images:
        with open(img_path, "rb") as image_file:
            encoded_img = base64.b64encode(image_file.read()).decode('utf-8')
            img_data = f"data:image/png;base64,{encoded_img}"
            html += f'<div style="margin-right: 10px;"><img src="{img_data}" style="height: 256px;"></div>'
    html += '</div>'
    display(HTML(html))


# Test

In [26]:
# Run the test script
!python test.py --dataroot ./datasets/MyDataset --name MyDataset_cyclegan --model cycle_gan --epoch latest --num_test 50 --checkpoints_dir /content/drive/MyDrive/GAN_project/checkpoints/class_model --results_dir /content/drive/MyDrive/GAN_project/checkpoints/class_model/test --batch_size 1 --no_dropout --direction AtoB


----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: /content/drive/MyDrive/GAN_project/checkpoints/class_model	[default: ./checkpoints]
                crop_size: 256                           
                 dataroot: ./datasets/MyDataset          	[default: None]
             dataset_mode: unaligned                     
                direction: AtoB                          
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
                init_gain: 0.02                          
                init_type: normal                        
                 input_nc: 3                             
                  isTrain: False                         	[default: None]
   

# Display imgs for testing

In [ ]:
import os
from IPython.display import display, HTML
import base64

# Path to the directory containing generated images
result_dir = '/content/drive/MyDrive/GAN_project/checkpoints/class_model/test/MyDataset_cyclegan/test_latest/images'

# Collect all image paths
image_paths = [os.path.join(result_dir, img_path) for img_path in sorted(os.listdir(result_dir)) if img_path.endswith('.png')]

# Check if the images exist and are accessible
for img_path in image_paths:
    if not os.path.exists(img_path):
        print(f"Image not found: {img_path}")
    else:
        print(f"Image found: {img_path}")

# HTML structure to display images
html_content = '<div style="display: flex; flex-wrap: wrap;">'

# Add images to HTML content
for i, img_path in enumerate(image_paths):
    with open(img_path, "rb") as image_file:
        encoded_image = base64.b64encode(image_file.read()).decode('utf-8')
        html_content += f'<div style="flex: 1 0 22%; margin: 5px;"><img src="data:image/png;base64,{encoded_image}" style="width: 100%; height: auto;"></div>'
    # Every 4 images, add a line break
    if (i + 1) % 4 == 0:
        html_content += '<div style="flex-basis: 100%; height: 0;"></div>'

html_content += '</div>'

# Display the images in HTML format
display(HTML(html_content))


# Summery of models

In [ ]:
!pip install torchsummary

In [54]:
import torch
from torchsummary import summary
from models import create_model

class CustomTestOptions:
    def __init__(self):
        self.dataroot =  '/content/drive/MyDrive/GAN_Project/To_Sumbit/dataset_class/'
        self.name = 'MyDataset_cyclegan'
        self.model = 'cycle_gan'
        self.checkpoints_dir = '/content/drive/MyDrive/GAN_project/checkpoints/class_model/'
        self.no_dropout = True
        self.num_threads = 0
        self.batch_size = 1
        self.serial_batches = True
        self.no_flip = True
        self.display_id = -1
        self.phase = 'train'
        self.epoch = 'latest'
        self.eval = True
        self.load_size = 256
        self.crop_size = 256
        self.gpu_ids = [0] if torch.cuda.is_available() else []

        self.isTrain = True
        self.input_nc = 3
        self.output_nc = 3
        self.ngf = 64
        self.ndf = 64
        self.netD = 'basic'
        self.netG = 'resnet_9blocks'
        self.preprocess = 'resize_and_crop'
        self.display_winsize = 256
        self.init_type = 'normal'
        self.init_gain = 0.02
        self.suffix = ''
        self.verbose = False
        self.norm = 'instance'
        self.dataset_mode = 'unaligned'
        self.direction = 'AtoB'
        self.lambda_identity = 0.5
        self.lambda_A = 10.0
        self.lambda_B = 10.0
        self.pool_size = 50
        self.lr = 0.0002
        self.beta1 = 0.5
        self.load_iter = 0
        self.continue_train = True
        self.epoch_count = 1
        self.niter = 100
        self.niter_decay = 100
        self.n_layers_D = 3
        self.gan_mode = 'lsgan'
        self.lr_policy = 'linear'
        self.n_epochs = 100
        self.n_epochs_decay = 100

# Create a new CustomTestOptions object
opt = CustomTestOptions()

# Initialize the model
model = create_model(opt)
model.setup(opt)


initialize network with normal
initialize network with normal
initialize network with normal
initialize network with normal
model [CycleGANModel] was created
loading the model from /content/drive/MyDrive/GAN_project/checkpoints/class_model/MyDataset_cyclegan/latest_net_G_A.pth
loading the model from /content/drive/MyDrive/GAN_project/checkpoints/class_model/MyDataset_cyclegan/latest_net_G_B.pth
loading the model from /content/drive/MyDrive/GAN_project/checkpoints/class_model/MyDataset_cyclegan/latest_net_D_A.pth
loading the model from /content/drive/MyDrive/GAN_project/checkpoints/class_model/MyDataset_cyclegan/latest_net_D_B.pth
---------- Networks initialized -------------
[Network G_A] Total number of parameters : 11.378 M
[Network G_B] Total number of parameters : 11.378 M
[Network D_A] Total number of parameters : 2.765 M
[Network D_B] Total number of parameters : 2.765 M
-----------------------------------------------


In [55]:
from torchsummary import summary

# Print summary for Generator A (G_A)
summary(model.netG_A, input_size=(3, 256, 256))  # Adjust input_size according to your model's input shape

# Print summary for Discriminator A (D_A)
summary(model.netD_A, input_size=(3, 256, 256))  # Adjust input_size accordingly

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ReflectionPad2d-1          [-1, 3, 262, 262]               0
            Conv2d-2         [-1, 64, 256, 256]           9,472
    InstanceNorm2d-3         [-1, 64, 256, 256]               0
              ReLU-4         [-1, 64, 256, 256]               0
            Conv2d-5        [-1, 128, 128, 128]          73,856
    InstanceNorm2d-6        [-1, 128, 128, 128]               0
              ReLU-7        [-1, 128, 128, 128]               0
            Conv2d-8          [-1, 256, 64, 64]         295,168
    InstanceNorm2d-9          [-1, 256, 64, 64]               0
             ReLU-10          [-1, 256, 64, 64]               0
  ReflectionPad2d-11          [-1, 256, 66, 66]               0
           Conv2d-12          [-1, 256, 64, 64]         590,080
   InstanceNorm2d-13          [-1, 256, 64, 64]               0
             ReLU-14          [-1, 256,